In [ ]:
%load_ext autoreload
%autoreload 2

### vis trajs

In [ ]:
import pickle
import matplotlib.pyplot as plt

In [ ]:
with open('../../../rozumarm-dataset/2023-07-27T16:14:09.956396.traj', 'rb') as f:
    traj = pickle.load(f)

In [ ]:
def process_sim_image(img):
    return img.transpose(1, 2, 0)


def process_real_image(img):
    return img.transpose(1, 2, 0)[..., ::-1]


def visualize_transition(t):
    text_prompt = t["text_prompt"]

    fig, axs = plt.subplots(3, 2)#, figsize=(12, 16))
    fig.subplots_adjust(hspace=0.3)

    fig.suptitle(text_prompt)

    
    axs[0, 0].imshow(process_sim_image(t["sim_before_action"]["rgb"]["top"]))
    axs[0, 1].imshow(process_sim_image(t["sim_after_action"]["rgb"]["top"]))
    axs[0, 1].set_title(f"success: {t['success_after_sim_swipe']}")

    axs[1, 0].imshow(process_real_image(t["real_before_action"]["rgb"]["top"]))
    axs[1, 1].imshow(process_real_image(t["real_after_action"]["rgb"]["top"]))
    axs[1, 1].set_title(f"success: {t['success_after_real_swipe']}, done: {t['done']}")

    # prompt assets
    swept_obj_images = t["prompt_assets"]["swept_obj"]["rgb"]
    axs[2, 0].imshow(process_sim_image(swept_obj_images["top"]))
    axs[2, 0].set_title("prompt.swept_obj.top")

    axs[2, 1].imshow(process_sim_image(swept_obj_images["front"]))
    axs[2, 1].set_title("prompt.swept_obj.front")

    for i in range(3):
        for j in range(2):
            axs[i, j].axis('off')
    return fig

fig = visualize_transition(traj["step_1"])

In [ ]:
fig.savefig('vima_failure_wrong_color.jpg', dpi=600, bbox_inches='tight')

---

In [ ]:
import math

import numpy as np
from scipy.spatial.transform import Rotation

from rozumarm_vima_utils.transform import (
    map_tf_repr,
    map_gripper_rf
)

In [ ]:
from rozumarm_vima_utils.robot import RozumArm, HOME_TCP_ANGLES, Z_PREP_LVL

In [ ]:
robot = RozumArm()

In [ ]:
# robot.api.open_gripper()
# robot.api.close_gripper()

### visit key points

In [ ]:
from rozumarm_vima_utils.transform import rf_tf_c2r


TABLE_FRAME_POINTS_FILEPATH = "assets/aruco_corners_top.npy"
ROZUM_FRAME_POINTS_FILEPATH = "assets/marker_points_in_rozum_rf_v1.npy"


key_points_crf = np.load(TABLE_FRAME_POINTS_FILEPATH)
key_points_crf = key_points_crf[:, :2]
key_points_rrf = np.load('assets/marker_points_in_rozum_rf_v2_synth.npy')

transformed_cam_points = np.stack([rf_tf_c2r(v) for v in key_points_crf])

In [ ]:
Z_BOT_BIAS = -0.085

i = 0
target_point = np.concatenate((key_points_rrf[i], [0.18 + 0.1 + Z_BOT_BIAS]))
robot._move_tcp(target_point, HOME_TCP_ANGLES)

In [ ]:
i = 3
target_point = np.concatenate((transformed_cam_points[i], [0.18 + 0.1 + Z_BOT_BIAS]))
robot._move_tcp(target_point, HOME_TCP_ANGLES)

### eef guiding

In [ ]:
pos = robot.api.get_position()
point = [getattr(pos.point, axis_name) for axis_name in ('x', 'y', 'z')]
rot = [getattr(pos.rotation, angle_name) for angle_name in ('roll', 'pitch', 'yaw')]

rot = [-math.pi, 0, 0]

# point[0] -= 0.0005
# point[1] += 0.005
# point[2] -= 0.01

robot._move_tcp(point, rot)

### test eef orientation during swipe

In [ ]:
vima_quat = robot.get_swipe_quat([0.0, 0.0], [0.1, 0.1])
vima_angles = Rotation.from_quat(vima_quat).as_euler('XYZ')
print(f'Desired rotation in VIMA-rf: {vima_angles}')

rozum_quat = map_tf_repr(map_gripper_rf(vima_quat))
rozum_angles = Rotation.from_quat(rozum_quat).as_euler('XYZ')
print(f'Desired rotation in Rozum-rf: {rozum_angles}')

### compute transform error

In [ ]:
import numpy as np
from rozumarm_vima_utils.transform import rf_tf_c2r

# v3 (synth alignment by 4.5 mm)
TABLE_FRAME_POINTS_FILEPATH = "../../assets/aruco_corners_top.npy"
ROZUM_FRAME_POINTS_FILEPATH = "../../assets/marker_points_in_rozum_rf_v2_synth.npy"

key_points_crf = np.load(TABLE_FRAME_POINTS_FILEPATH)
key_points_crf = key_points_crf[:, :2]
key_points_rrf = np.load(ROZUM_FRAME_POINTS_FILEPATH)

transformed_cam_points = np.stack([rf_tf_c2r(p) for p in key_points_crf])

with np.printoptions(precision=2, suppress=True):
    errors = np.linalg.norm(key_points_rrf - transformed_cam_points, axis=1)
    print(f"(mm): {errors * 1e3}")